In [1]:
training_data=[
    ['Green',3,'Mango'],
    ['Yellow',3,'Mango'],
    ['Red',1,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon'],
]

In [2]:
header=["Color","Diameter","Label"]

In [3]:
def unique_vals(rows,col):
    return set([row[col]for row in rows])

In [4]:
unique_vals(training_data,0)

{'Green', 'Red', 'Yellow'}

In [5]:
unique_vals(training_data,1)

{1, 3}

In [6]:
unique_vals(training_data,2)

{'Grape', 'Lemon', 'Mango'}

In [7]:
"""Counts the no of each type of example in a dataset"""
def class_counts(rows):
    counts={} #a dict of label-> count.
    for row in rows: 
        # in our dataset format, the label is always the last col
        label=row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts

In [8]:
class_counts(training_data)

{'Mango': 2, 'Grape': 2, 'Lemon': 1}

In [9]:
"""Test if a value is numeric."""
def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

In [10]:
#is_numeric("red") 
#is_numeric(3)

In [11]:
class Question:
    """A Question is used to partition a dataset.
    This class just records a 'column number' (e.g., 0 for color) and a
    'coloumn value' (e.g., Green). The match method is used to compare 
    the feature value in an example to the feature value stored in the 
    Question. See the demo below.
    """
    def __init__(self,column,value):
        self.column=column
        self.value=value

    def match(self,example):
        #compare the feature value in eg to feature value in this question
        val=example[self.column]
        if is_numeric(val):
            return val>=self.value
        else:
            return val==self.value

    def __repr__(self):
        #helper method to print the Question in a readable format
        condition="=="
        if is_numeric(self.value):
            condition=">="
        return "Is %s %s %s?"%(header[self.column],condition,str(self.value))        

In [12]:
q=Question(0,'Green')
print(q)

Is Color == Green?


In [13]:
print([q.match(row) for row in training_data])

[True, False, False, False, False]


In [14]:
q2=Question(1,3)
print(q2)

Is Diameter >= 3?


In [15]:
print([q2.match(row) for row in training_data])

[True, True, False, False, True]


In [16]:
def partition(rows,question):
    """Partitions a dataset.
    For each row in the dataset,check if it matches the question. If so,
    add it to 'true rows',otherwise, add it to 'false rows'."""
    true_rows,false_rows=[],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows       

In [17]:
#lets partition the training data based on whether rows are red.
true_rows,false_rows=partition(training_data,Question(0,'Red'))
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [18]:
false_rows

[['Green', 3, 'Mango'], ['Yellow', 3, 'Mango'], ['Yellow', 3, 'Lemon']]

In [19]:
def gini(rows):
    #calculate gini impurity for a list of row.
    counts=class_counts(rows)
    impurity=1
    for lbl in counts:
        prob_of_lbl=counts[lbl]/float(len(rows))
        impurity-=prob_of_lbl**2
    return impurity   

In [20]:
def info_gain(left,right,current_uncertainty):
    """Information Gain
    The uncertainity of the starting node,minus the weighted impurity of
    two child nodes."""
    p=float(len(left))/(len(left)+len(right))
    return current_uncertainity-p*gini(left)-(1-p)*gini(right)

In [21]:
current_uncertainity=gini(training_data) #calculate uncertainity
#how much info do we gain by partitioning on 'green'?
true_rows,false_rows=partition(training_data,Question(0,'Green'))
info_gain(true_rows,false_rows,current_uncertainity)

0.1399999999999999

In [22]:
#what if we paritioned on 'red' instead?
true_rows,false_rows=partition(training_data,Question(0,'Red'))
info_gain(true_rows,false_rows,current_uncertainity)

0.37333333333333324

In [23]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
       and calculating the info gain."""
    best_gain=0
    best_question=None
    current_uncertainity=gini(rows)
    n_features=len(rows[0])-1 #no. of rows
    for col in range(n_features):
        values=set([row[col] for row in rows])
        for val in values:
            question =Question(col,val)
            true_rows,false_rows=partition(rows,question)
            if len(true_rows)==0 or len(false_rows)==0:
                continue # skip if it doesnt divide the dataset
            gain=info_gain(true_rows,false_rows,current_uncertainity)

            if gain>= best_gain:
                best_gain,best_question=gain,question
    return best_gain,best_question           

In [24]:
best_gain,best_question=find_best_split(training_data)

In [25]:
class Leaf:
    """A leaf node classifies data.
    This holds a dict of class (e.g., mango)-> no. of times it apears in
    the rows from the trianig data that reach this leaf."""
    def __init__(self,rows):
        self.predictions=class_counts(rows)
        

In [26]:
class Decision_Node:
    """A decision node asks a question.
    This holds a reference to the question,and to the two child nodes."""
    def __init__(self,question,true_branch,false_branch):
        self.question=question
        self.true_branch=true_branch
        self.false_branch=false_branch

In [27]:
def build_tree(rows):
    # partition on each unique attribute
    # calculate the info gain
    # return question that produces the highest gain
    gain, question=find_best_split(rows)

    #base case: no further info gain
    #since we can ask no further question
    #we'll return a leaf
    if gain==0:
        return Leaf(rows)

    #if we reach here, we've found a useul feature/value to partition on.
    true_rows,false_rows=partition(rows,question)

    #recursively build the true and false branch.
    true_branch=build_tree(true_rows)
    false_branch=build_tree(false_rows)

    #return a Question node.
    #This records the best feature/value to ask at this point, depending on ans.
    return Decision_Node(question,true_branch,false_branch)
    

In [28]:
def print_tree(node,spacing=""):
    # worlds most elegant tree printing function.
    # Base case: we've reached a leaf
    if isinstance(node,Leaf):
        print(spacing+"Predict",node.predictions)
        return
    #print the question at this node
    print(spacing + str(node.question))

    #call this func recursively on the true branch
    print(spacing+'--> True:')
    print_tree(node.true_branch,spacing+" ")

    #call this func recusively on the false branch
    print(spacing + '-->False:')
    print_tree(node.false_branch,spacing+" ")

In [29]:
def classify(row,node):
    #basecase: we've reached a leaf
    if isinstance(node,Leaf):
        return node.predictions

    #decide wheher to follow the true or false branch
    #compare the feature/value stored in a node, to eg we're considering
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)

In [30]:
my_tree = build_tree(training_data)
print(classify(training_data[0], my_tree))  

{'Mango': 1}


In [31]:
def print_leaf(counts):
    """Print the predictions at a leaf."""
    total=sum(counts.values())*1.0
    probs={}
    for lbl in counts.keys():
        probs[lbl]=str(int(counts[lbl]/total*100))+"%"
    return probs

In [32]:
print_leaf(classify(training_data[0],my_tree))

{'Mango': '100%'}

In [33]:
if __name__=='__main__':
    my_tree=build_tree(training_data)
    print_tree(my_tree)
    testing_data=[
    ['Green',3,'Mango'],
    ['Green',4,'Mango'],
    ['Red',2,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon'],
]
    for row in testing_data: 
        print("Actual: %s. Predicted: %s" %(row[-1],print_leaf(classify(row,my_tree))))

Is Diameter >= 3?
--> True:
 Is Color == Yellow?
 --> True:
  Predict {'Mango': 1, 'Lemon': 1}
 -->False:
  Predict {'Mango': 1}
-->False:
 Predict {'Grape': 2}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
